In [1]:
## Import the necessary libraries
import sqlite3
import pandas as pd
import os
import time
import numpy as np

In [2]:
# Paths
BASE_DIR = os.path.abspath("..")                      # github_repo/
DATA_DIR = os.path.join(BASE_DIR, "data", "raw")       # github_repo/data/raw
DB_PATH = os.path.join(BASE_DIR, "insurance.db")       # github_repo/insurance.db

In [3]:
# Connect back to the database
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

In [4]:
# Execute the query to just select my table from the database
start = time.time()
df=pd.read_sql("SELECT * FROM combined_data_pre_cleaning",conn)
end = time.time()
conn.close()

In [5]:
# Confirm the size of my df is what I would expect
print(f"\nQuery returned {df.shape[0]:,} rows x {df.shape[1]} columns in {end - start:.2f} seconds")
pd.set_option('display.max_columns', None)
df.head()


Query returned 2,280,250 rows x 22 columns in 21.11 seconds


,INDIVIDUAL_ID,ADDRESS_ID,CURR_ANN_AMT,DAYS_TENURE,CUST_ORIG_DATE,age_in_years,LATITUDE,LONGITUDE,STREET_ADDRESS,CITY,STATE,COUNTY,Churn,income,ACCT_SUSPD_DATE,HAS_CHILDREN,LENGTH_OF_RESIDENCE,marital_status,HOME_MARKET_VALUE,HOME_OWNER,COLLEGE_DEGREE,GOOD_CREDIT
0,NaN,NaN,660.852375,2810.0,2015-03-24,NaN,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,NaN
1,2.213000e+11,5.213000e+11,1194.050321,6291.0,2005-09-11,66.387,32.964555,-96.819410,312 Austin Plains,Dallas,TX,Dallas,NaN,80372.176,None,0.0,6.801,None,500000 - 749999,0.0,1.0,1.0
2,2.213000e+11,5.213000e+11,837.936100,4496.0,2010-08-11,58.968,32.964555,-96.819410,312 Austin Plains,Dallas,TX,Dallas,0.0,125000.000,None,1.0,2.000,Single,500000 - 749999,1.0,0.0,1.0
3,2.213000e+11,5.213000e+11,1141.116276,6291.0,2005-09-11,NaN,32.600787,-97.101485,USNS Lee,Mansfield,TX,Tarrant,NaN,NaN,None,NaN,NaN,None,None,NaN,NaN,NaN
4,2.213000e+11,5.213000e+11,695.119342,6291.0,2005-09-11,64.641,32.563342,-97.058826,3622 Robert Ridges Suite 152,Mansfield,TX,Tarrant,0.0,70000.000,None,1.0,7.000,Married,125000 - 149999,1.0,1.0,1.0


In [6]:
# Convert the dates to type datetime
# df['ACCT_SUSPD_DATE'] = pd.to_datetime(df['ACCT_SUSPD_DATE'],errors='coerce')

In [7]:
## Logic applied here: if there is a termination date, then we know there is churn happening
# df.loc[df['Churn'].isna() & df['ACCT_SUSPD_DATE'].notna(),'Churn']=1

In [8]:
# Let's confirm the data types of each of my variables of interest
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280250 entries, 0 to 2280249
Data columns (total 22 columns):
 #   Column               Dtype  
---  ------               -----  
 0   INDIVIDUAL_ID        float64
 1   ADDRESS_ID           float64
 2   CURR_ANN_AMT         float64
 3   DAYS_TENURE          float64
 4   CUST_ORIG_DATE       object 
 5   age_in_years         float64
 6   LATITUDE             float64
 7   LONGITUDE            float64
 8   STREET_ADDRESS       object 
 9   CITY                 object 
 10  STATE                object 
 11  COUNTY               object 
 12  Churn                float64
 13  income               float64
 14  ACCT_SUSPD_DATE      object 
 15  HAS_CHILDREN         float64
 16  LENGTH_OF_RESIDENCE  float64
 17  marital_status       object 
 18  HOME_MARKET_VALUE    object 
 19  HOME_OWNER           float64
 20  COLLEGE_DEGREE       float64
 21  GOOD_CREDIT          float64
dtypes: float64(14), object(8)
memory usage: 382.7+ MB


In [9]:
# Let's make sure that there are no duplicate counts for individual ID's
quantity = df['INDIVIDUAL_ID'].value_counts()
duplicates_in_name = quantity[quantity > 1]
print("Table of 'Individual_ID' duplicate column:\n", duplicates_in_name)

duplicate_rows_count = df[df.duplicated(subset=["INDIVIDUAL_ID"],keep=False)].shape[0]
print("Number of dupliacted customers: ",duplicate_rows_count)

Table of 'Individual_ID' duplicate column:
 Series([], Name: count, dtype: int64)
Number of dupliacted customers:  0


The cell above shows that there are no duplicates in the individual_id column which is good to set this as the true value for all of our modeling

In [10]:
# I want to get an idea of how many duplicates there are for address id's
quantity = df['ADDRESS_ID'].value_counts()
duplicates_in_name = quantity[quantity > 1]
print("Table of 'ADDRESS_ID' duplicate column:\n", duplicates_in_name)

duplicate_rows_count = df[df.duplicated(subset=["ADDRESS_ID"],keep=False)].shape[0]
print("Number of duplicated addresses: ","{:,}".format(duplicate_rows_count))

Table of 'ADDRESS_ID' duplicate column:
 ADDRESS_ID
5.213006e+11    468
5.213005e+11    442
5.213009e+11    336
5.213002e+11    332
5.213004e+11    308
               ... 
5.213010e+11      2
5.213003e+11      2
5.213000e+11      2
5.213000e+11      2
5.213000e+11      2
Name: count, Length: 394769, dtype: int64
Number of duplicated addresses:  1,138,397


As we can see, there are many duplicated addresses with multiple individual id's living at the same address

In [11]:
# I'm going to create a helper function to help me see how many missing values I have in my code
def missing_summary(df):
    pd.set_option('display.max_columns',None)
    print(pd.DataFrame({
        'Missing Values': df.isna().sum(),
        'Percentage': (df.isna().sum() / len(df))*100
    }))

# This will be a helper function that I use for filling in the na's for home_market_value with the median
def parse_home_value(val):
    if pd.isna(val):
        return np.nan
    val = str(val).strip()
    if val.upper() == "N/A":
        return np.nan
    if " - " in val:
        lo, hi = val.split(" - ")
        return (float(lo) + float(hi)) / 2
    try:
        return float(val)
    except ValueError:
        return np.nan  
    
# This will be a helper function for deleting all corresponding X/y indexes for rows with n/a's representing < 1 % of that column
def drop_na_rows(X,y,column_name):
    mask = X[column_name].notna()
    return X[mask].copy(), y[mask].copy()

In [12]:
df = df.dropna(subset=['CURR_ANN_AMT']) # Drop the rows without curr_ann_amt
df = df.dropna(subset=['INDIVIDUAL_ID']) # Drop the rows without individual_id's (Only 1 row for some reason)

# Calling out the current annual amount as our predictor target
y = df[['CURR_ANN_AMT']]

# Specifying our features that we will be using for model fitting
features = ['DAYS_TENURE','age_in_years','LATITUDE','LONGITUDE','CITY','income','HAS_CHILDREN','LENGTH_OF_RESIDENCE','marital_status','HOME_MARKET_VALUE','HOME_OWNER','COLLEGE_DEGREE','GOOD_CREDIT']
X = df[features] 

# Take a peak at how the features matrix looks 
pd.set_option('display.max_columns',None)
missing_summary(X)
X.head()

                     Missing Values  Percentage
DAYS_TENURE                       0    0.000000
age_in_years                 167759    7.357047
LATITUDE                     471474   20.676426
LONGITUDE                    471474   20.676426
CITY                          17006    0.745796
income                       167759    7.357047
HAS_CHILDREN                 167759    7.357047
LENGTH_OF_RESIDENCE          167759    7.357047
marital_status               599389   26.286121
HOME_MARKET_VALUE            357953   15.697979
HOME_OWNER                   167759    7.357047
COLLEGE_DEGREE               167759    7.357047
GOOD_CREDIT                  167759    7.357047


,DAYS_TENURE,age_in_years,LATITUDE,LONGITUDE,CITY,income,HAS_CHILDREN,LENGTH_OF_RESIDENCE,marital_status,HOME_MARKET_VALUE,HOME_OWNER,COLLEGE_DEGREE,GOOD_CREDIT
1,6291.0,66.387,32.964555,-96.819410,Dallas,80372.176,0.0,6.801,None,500000 - 749999,0.0,1.0,1.0
2,4496.0,58.968,32.964555,-96.819410,Dallas,125000.000,1.0,2.000,Single,500000 - 749999,1.0,0.0,1.0
3,6291.0,NaN,32.600787,-97.101485,Mansfield,NaN,NaN,NaN,None,None,NaN,NaN,NaN
4,6291.0,64.641,32.563342,-97.058826,Mansfield,70000.000,1.0,7.000,Married,125000 - 149999,1.0,1.0,1.0
5,549.0,34.639,33.006625,-97.203735,Roanoke,87500.000,0.0,3.000,Single,100000 - 124999,1.0,0.0,1.0


Notes on missing data:
-The AGE_IN_YEARS would be expected to be 0 since it is from the customer table (the primary table) but there are just nulls in their anyways despiste a COALESCE from the autoinsurance_churn file
-The ACCT_SUSPD_DATE is lean since the termination file is lean on data but that predictor will not be used in modeling anyways

In [13]:
# Here will be my block of code for correcting out the na's and range values given for home_market_value
X["HOME_MARKET_VALUE"] = X["HOME_MARKET_VALUE"].apply(parse_home_value)
overall_home_median = X["HOME_MARKET_VALUE"].median()
print("Overall Home Median :", f"${overall_home_median:,.2f}") # This will print out what the overall home median value is
X["HOME_MARKET_VALUE"] = X["HOME_MARKET_VALUE"].fillna(overall_home_median)

# We need to compute city-level medians for LATITUDE and LONGITUDE
city_medians = X.groupby("CITY")[["LATITUDE","LONGITUDE"]].median()

# Map back these medians
X["lat_median"] = X["CITY"].map(city_medians["LATITUDE"])
X["long_median"] = X["CITY"].map(city_medians["LONGITUDE"])

# Fill missing lat/long wiht city medians 
X["LATITUDE"] = X["LATITUDE"].fillna(X["lat_median"])
X["LONGITUDE"] = X["LONGITUDE"].fillna(X["long_median"])

# We can drop city now since we no longer need it
X = X.drop(columns=['CITY'])

# For missing marital_status we are going to put all the n/a's as 0's and add another column for marital_status_missing marked with a 1
X['marital_status_missing'] = X['marital_status'].isna().astype(int)
X['marital_status'] = X['marital_status'].fillna(0)
X['marital_status'] = X['marital_status'].replace({"Single":0,"Married":1})

## For the missing and n/a values for 'Has_Children', 'Home_Owner', 'College_Degree', and 'Good_Credit': since the missing values only represent 7% of the dataset, let's put in a "Is_missing" category to see if missingness correlates with premiums.
for col in ['HAS_CHILDREN','HOME_OWNER','COLLEGE_DEGREE','GOOD_CREDIT']:
    X[col + '_missing'] = X[col].isna().astype(int)
    X[col] = X[col].fillna(0)

## For age, income, and length of residence: fill in the missing values with the median values
for col in ['age_in_years','income','LENGTH_OF_RESIDENCE']:
    X[col + '_missing'] = X[col].isna().astype(int)
    X[col] = X[col].fillna(X[col].median())

X, y = drop_na_rows(X,y,"marital_status")
X, y = drop_na_rows(X,y,"LATITUDE")
X, y = drop_na_rows(X,y,"LONGITUDE")
X, y = drop_na_rows(X,y,"lat_median")
X, y = drop_na_rows(X,y,"long_median")



# Take a peak at how the features matrix looks 
pd.set_option('display.max_columns',None)
missing_summary(X)
X.head()

C:\Users\JDECKE46\AppData\Local\Temp\ipykernel_40812\886487437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["HOME_MARKET_VALUE"] = X["HOME_MARKET_VALUE"].apply(parse_home_value)
C:\Users\JDECKE46\AppData\Local\Temp\ipykernel_40812\886487437.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["HOME_MARKET_VALUE"] = X["HOME_MARKET_VALUE"].fillna(overall_home_median)


Overall Home Median : $112,499.50


C:\Users\JDECKE46\AppData\Local\Temp\ipykernel_40812\886487437.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["lat_median"] = X["CITY"].map(city_medians["LATITUDE"])
C:\Users\JDECKE46\AppData\Local\Temp\ipykernel_40812\886487437.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["long_median"] = X["CITY"].map(city_medians["LONGITUDE"])
C:\Users\JDECKE46\AppData\Local\Temp\ipykernel_40812\886487437.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


                             Missing Values  Percentage
DAYS_TENURE                               0         0.0
age_in_years                              0         0.0
LATITUDE                                  0         0.0
LONGITUDE                                 0         0.0
income                                    0         0.0
HAS_CHILDREN                              0         0.0
LENGTH_OF_RESIDENCE                       0         0.0
marital_status                            0         0.0
HOME_MARKET_VALUE                         0         0.0
HOME_OWNER                                0         0.0
COLLEGE_DEGREE                            0         0.0
GOOD_CREDIT                               0         0.0
lat_median                                0         0.0
long_median                               0         0.0
marital_status_missing                    0         0.0
HAS_CHILDREN_missing                      0         0.0
HOME_OWNER_missing                        0     

,DAYS_TENURE,age_in_years,LATITUDE,LONGITUDE,income,HAS_CHILDREN,LENGTH_OF_RESIDENCE,marital_status,HOME_MARKET_VALUE,HOME_OWNER,COLLEGE_DEGREE,GOOD_CREDIT,lat_median,long_median,marital_status_missing,HAS_CHILDREN_missing,HOME_OWNER_missing,COLLEGE_DEGREE_missing,GOOD_CREDIT_missing,age_in_years_missing,income_missing,LENGTH_OF_RESIDENCE_missing
1,6291.0,66.387,32.964555,-96.819410,80372.176,0.0,6.801,0,624999.5,0.0,1.0,1.0,32.807488,-96.792844,1,0,0,0,0,0,0,0
2,4496.0,58.968,32.964555,-96.819410,125000.000,1.0,2.000,0,624999.5,1.0,0.0,1.0,32.807488,-96.792844,0,0,0,0,0,0,0,0
3,6291.0,55.444,32.600787,-97.101485,80372.176,0.0,6.801,0,112499.5,0.0,0.0,0.0,32.586145,-97.127821,1,1,1,1,1,1,1,1
4,6291.0,64.641,32.563342,-97.058826,70000.000,1.0,7.000,1,137499.5,1.0,1.0,1.0,32.586145,-97.127821,0,0,0,0,0,0,0,0
5,549.0,34.639,33.006625,-97.203735,87500.000,0.0,3.000,0,112499.5,1.0,0.0,1.0,32.996025,-97.212322,0,0,0,0,0,0,0,0


For missing values, let's implement a "was_missing" feature so the model can learn if missingness correlates with premiums

In [14]:
conn = sqlite3.connect(DB_PATH)

y.to_sql("y",conn,if_exists="replace",index=False)
X.to_sql("X",conn,if_exists="replace",index=False)

conn.close()